In [382]:
from collections import namedtuple
import numpy as np
import pandas as pd
from pathlib import Path

# Display more rows and get rid of the margins
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Display mulitiple values from each cell
# https://stackoverflow.com/a/42476224
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Define paths and functions

In [383]:
abcd_table_path = './ABCDstudyDEAP2.0/'
data_dir = Path('data')

In [384]:
def load_abcd_table(path):
    table = pd.read_csv(path, skiprows=[1], header=0, sep='\t')
    labels = pd.read_csv(path, nrows=1, header=0, sep='\t')
    labels = labels.T.reset_index().rename(columns={'index':'name', 0:'doc'})
    return table, labels

def append_abcd_table(path, table=None, label=None, on=None):
    if table is None:
        return load_abcd_table(path)
    if on is None:
        on = ['subjectkey', 'interview_date', 'visit', 'lmt_run']
    new_table, new_label = load_abcd_table(path)
    # Find drop common columns that aren't in the set we're merging on
    drop_cols = label.loc[label.doc.isin(new_label.doc) & ~label.name.isin(on),'name'].values
    new_table.drop(drop_cols, axis=1, inplace=True)
    new_label = new_label[~new_label.name.isin(drop_cols) & ~new_label.name.isin(on)]
    
    # Get table shapes for error checking
    ts = table.shape
    nts = new_table.shape
    ls = label.shape
    nls = new_label.shape
    
    # Merge the tables
    table = table.merge(new_table, how='outer',on=on)
    
    # Check for success
    assert table.shape[0] == ts[0] == nts[0]
    assert table.shape[1] == (ts[1] + nts[1] - len(on))
    
    # Merge labels
    label = pd.concat([label, new_label]).reset_index(drop=True)
    
    assert label.shape[0] == table.shape[1]
    
    return table, label

def load_task_melt_contrasts(paths, contrasts, task_name):
    task_t = None
    task_l = None
    for pp in paths:
        task_t, task_l = append_abcd_table(pp, table=task_t, label=task_l)
        
    # Rearrange columns
    new_col_order = pd.concat([task_l.name[~task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts)],
                               task_l.name[task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts)]]).values
    task_t = task_t.loc[:,new_col_order]
    task_l = task_l.set_index('name').T.loc[:,new_col_order].T.reset_index()
    
    # Manipulate labels to get more information about them
    if task_name == 'nb':
        task_l.loc[task_l.doc.str.split('for ').str[1].str.split(' in').str[0].isin(contrasts), 'contrast'] = task_l.doc.str.split('for ').str[1].str.split(' in').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_code'] = task_l.loc[pd.notnull(task_l.contrast),'name'].str.split('x').str[-1]
        task_l.loc[pd.notnull(task_l.contrast),'hemisphere'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('g').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_num'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('p').str[-1].astype(int)
        task_meta_cols = task_l.name[~task_l.doc.str.split('for ').str[1].str.split(' in').str[0].isin(contrasts)]
    else:
        task_l.loc[task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts), 'contrast'] = task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_code'] = task_l.loc[pd.notnull(task_l.contrast),'name'].str.split('x').str[-1]
        task_l.loc[pd.notnull(task_l.contrast),'hemisphere'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('g').str[0]
        task_l.loc[pd.notnull(task_l.contrast),'roi_num'] = task_l.loc[pd.notnull(task_l.contrast),'roi_code'].str.split('p').str[-1].astype(int)
        task_meta_cols = task_l.name[~task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0].isin(contrasts)]

    task_t_melt = []
    for contrast in contrasts:
        tmpdf = task_t.loc[:,task_meta_cols]
        tmpdf['contrast'] = contrast
        if task_name == 'nb':
            contrast_cols = task_l.name[task_l.doc.str.split('for ').str[1].str.split(' in').str[0] == contrast].values
        else:
            contrast_cols = task_l.name[task_l.doc.str.split('for ').str[1].str.split(' contrast in').str[0] == contrast].values
        contrast_df = task_t.loc[:,contrast_cols]
        contrast_df.columns = [task_l.loc[task_l.name==cc,'roi_code'].values[0] for cc in contrast_df.columns.values]
        task_t_melt.append(pd.concat([tmpdf,contrast_df], axis=1))
    task_t_melt = pd.concat(task_t_melt)
    task_t_melt['task'] = task_name
    return task_t_melt, task_l

# Prepare data
## Get sex, handedness, age and device data for balancing

In [385]:
# Get Handedness data
ehis_path = abcd_table_path + 'abcd_ehis01.txt'
ehis = pd.read_csv(ehis_path, skiprows=[1], header=0, sep='\t')
ehis_labels = pd.read_csv(ehis_path, nrows = 1, header = 0, sep='\t')
ehis_labels = ehis_labels.T.reset_index().rename(columns={'index':'name', 0:'doc'})

In [386]:
ehis_labels

,name,doc
0,collection_id,collection_id
1,abcd_ehis01_id,abcd_ehis01_id
2,dataset_id,dataset_id
3,subjectkey,The NDAR Global Unique Identifier (GUID) for research subject
4,src_subject_id,Subject ID how it's defined in lab/project
5,interview_date,Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY
6,interview_age,Age in months at the time of the interview/test/sampling/imaging.
7,gender,Sex of the subject
8,eventname,The event name for which the data was collected
9,ehi_1b,Writing


In [387]:
# Get site data
site_df = pd.read_csv(abcd_table_path + 'abcd_lt01.txt',
                      skiprows = [1],
                      header = 0,
                      sep='\t')

# Get device data
device_df = pd.read_csv(abcd_table_path + 'abcd_mri01.txt',
                        skiprows = [1],
                        header = 0,
                        sep='\t')

# Put the handedness, age, and sex information together with the device data

bal_site_fields = ['abcd_lt01_id',
                   'subjectkey',
                   'eventname',
                   'site_id_l'
                   ]
bal_ehis_fields = ['collection_id',
                   'dataset_id',
                   'subjectkey',
                   'interview_date',
                   'interview_age',
                   'gender',
                   'ehi_ss_score',
                   'eventname'
                  ]
bal_device_fields = ['subjectkey',
                     'eventname',
                     'mri_info_visitid',
                     'mri_info_manufacturer',
                     'mri_info_manufacturersmn',
                     'mri_info_deviceserialnumber',
                     'mri_info_magneticfieldstrength',
                     'mri_info_softwareversion'
                    ]

# Add to this once we know what the event name is for the second imaging visit
imaging_events = ['baseline_year_1_arm_1']

for_balancing = ehis.loc[:, bal_ehis_fields].merge(site_df.loc[:,bal_site_fields],
                            how='outer',
                            on=['subjectkey', 'eventname'],
                            indicator='site_indicator').merge(device_df.loc[:,bal_device_fields],
                            how='outer',
                            on=['subjectkey', 'eventname'],
                            indicator='device_indicator')

# find out how many subjects with imaging visit don't have imaging data listed in abcd_mri01
noimaging = for_balancing.groupby(['device_indicator'])[['subjectkey']].count().loc['both', 'subjectkey']
print(f"{noimaging} subjects don't have data listed in abcd_mri who should have had an imaging visit")

# drop the subjects without imaging data
for_balancing = for_balancing.query("device_indicator == 'both'")

# make sure both merges worked
assert for_balancing.groupby(['site_indicator'])[['subjectkey']].count().loc['both', 'subjectkey'] == len(device_df)
assert for_balancing.groupby(['device_indicator'])[['subjectkey']].count().loc['both', 'subjectkey'] == len(device_df)

for_balancing = for_balancing.drop(['site_indicator', 'device_indicator'], axis=1)

11755 subjects don't have data listed in abcd_mri who should have had an imaging visit


## Assemble rs-fMRI dataframe

In [388]:
# Between network table
bn_t, bn_l = load_abcd_table(abcd_table_path + 'abcd_betnet02.txt')
# Network to subcortical table
ns_t, ns_l = load_abcd_table(abcd_table_path + 'mrirscor02.txt')
# See which columns are in both tables
bn_l.loc[bn_l.doc.isin(ns_l.doc),:]

,name,doc
0,collection_id,collection_id
2,dataset_id,dataset_id
3,subjectkey,The NDAR Global Unique Identifier (GUID) for research subject
4,src_subject_id,Subject ID how it's defined in lab/project
5,interview_date,Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY
6,interview_age,Age in months at the time of the interview/test/sampling/imaging.
7,gender,Sex of the subject
8,eventname,The event name for which the data was collected
9,rsfmri_cor_network.gordon_visitid,Visit name
10,rsfmri_cor_network.gordon_tr,Pulse repetition time in seconds


In [389]:
# We'll merge on subjectkey and interview_date, filter out the other columns from ns_t before merging
ns_t.drop(ns_l.loc[ns_l.doc.isin(bn_l.doc),'name'].drop([3,5]).values,
          axis=1,
          inplace=True)

con = bn_t.merge(ns_t,
                 how='outer',
                 on=['subjectkey','interview_date'])

con_l = pd.concat([bn_l, 
                   ns_l[~ns_l.name.isin(bn_l.loc[bn_l.doc.isin(ns_l.doc),'name'].values)]]).reset_index(drop=True)

In [390]:
# Merge in balancing variables
con = con.merge(for_balancing.loc[:,['subjectkey','eventname','ehi_ss_score', 'site_id_l', 'mri_info_manufacturer',
       'mri_info_manufacturersmn', 'mri_info_deviceserialnumber', 'mri_info_magneticfieldstrength', 'mri_info_softwareversion']], how='left',
          on=['subjectkey','eventname'], indicator=True)

#con_l = con_l.append(pd.DataFrame([{'name':'ehi_ss_score','doc':'Handedness score rating'}]))

# add id for site, scanner, software
con['unique_scanner'] = con.site_id_l + '__' + con.mri_info_deviceserialnumber + '__' + con.mri_info_softwareversion

In [391]:
con.groupby(['_merge'])[['subjectkey']].count()
con.drop('_merge', 1, inplace=True)

,subjectkey
_merge,
left_only,0
right_only,0
both,10955


In [392]:
# Get a list of Metric columns
con_meta_cols = ['collection_id', 'dataset_id', 'subjectkey',
            'src_subject_id', 'interview_date', 'interview_age',
            'gender', 'event_name', 'visit', 'rsfm_tr', 'eventname',
            'rsfm_nreps', 'rsfm_numtrs', 'pipeline_version',  'scanner_manufacturer_pd',
            'scanner_type_pd','deviceserialnumber', 'magnetic_field_strength',
            'procdate', 'collection_title', 'promoted_subjectkey',
            'study_cohort_name','ehi_ss_scoreb', 'qc_ok', 'site_id_l', 'mri_info_manufacturer',
       'mri_info_manufacturersmn', 'mri_info_deviceserialnumber', 'mri_info_magneticfieldstrength', 'mri_info_softwareversion']
con_metric_cols = con_l.loc[~con_l.name.isin(con_meta_cols), 'name'].values


# pull out the roi and network names 

con_l['from_network'] = con_l.loc[con_l.name.isin(con_metric_cols),'doc'].str.split('between ').str[1].str.split('network').str[0]
con_l['to'] = con_l.loc[con_l.name.isin(con_metric_cols),'doc'].str.split('between ').str[1].str.split('network').str[1].str.split(' and').str[1].str.replace('ASEG ROI', '')
con_l['roi'] = con_l.loc[con_l.name.isin(con_metric_cols),'doc'].str.split('and ASEG ROI').str[1].str.strip()

con_l['from_network'].unique()
con_l.roi.unique()

array([nan, 'auditory ', 'cingulo-opercular ', 'cingulo-parietal ',
       'default ', 'dorsal attention ', 'fronto-parietal ', '"none" ',
       'retrosplenial temporal ', 'sensorimotor hand ',
       'sensorimotor mouth ', 'salience ', 'ventral attention ',
       'visual '], dtype=object)

array([nan, 'left-cerebellum-cortex', 'left-thalamus-proper',
       'left-caudate', 'left-putamen', 'left-pallidum', 'brain-stem',
       'left-hippocampus', 'left-amygdala', 'left-accumbens-area',
       'left-ventraldc', 'right-cerebellum-cortex',
       'right-thalamus-proper', 'right-caudate', 'right-putamen',
       'right-pallidum', 'right-hippocampus', 'right-amygdala',
       'right-accumbens-area', 'right-ventraldc'], dtype=object)

In [393]:
con.head()

,collection_id,abcd_betnet02_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,gender,eventname,rsfmri_cor_network.gordon_visitid,rsfmri_cor_network.gordon_tr,rsfmri_cor_network.gordon_numtrs,rsfmri_cor_network.gordon_nvols,rsfmri_cor_network.gordon_subthresh.nvols,rsfmri_cor_network.gordon_subthresh.contig.nvols,rsfmri_cor_network.gordon_ntpoints,rsfmri_cor_network.gordon_mean.motion,rsfmri_cor_network.gordon_max.motion,rsfmri_cor_network.gordon_mean.trans,rsfmri_cor_network.gordon_max.trans,rsfmri_cor_network.gordon_mean.rot,rsfmri_cor_network.gordon_max.rot,rsfmri_cor_network.gordon_auditory_network.gordon_auditory,rsfmri_cor_network.gordon_auditory_network.gordon_cingulooperc,rsfmri_cor_network.gordon_auditory_network.gordon_cinguloparietal,rsfmri_cor_network.gordon_auditory_network.gordon_default,rsfmri_cor_network.gordon_auditory_network.gordon_dorsalattn,rsfmri_cor_network.gordon_auditory_network.gordon_frontoparietal,rsfmri_cor_network.gordon_auditory_network.gordon_none,rsfmri_cor_network.gordon_auditory_network.gordon_retrosplenialtemporal,rsfmri_cor_network.gordon_auditory_network.gordon_smhand,rsfmri_cor_network.gordon_auditory_network.gordon_smmouth,rsfmri_cor_network.gordon_auditory_network.gordon_salience,rsfmri_cor_network.gordon_auditory_network.gordon_ventralattn,rsfmri_cor_network.gordon_auditory_network.gordon_visual,rsfmri_cor_network.gordon_cingulooperc_network.gordon_auditory,rsfmri_cor_network.gordon_cingulooperc_network.gordon_cingulooperc,rsfmri_cor_network.gordon_cingulooperc_network.gordon_cinguloparietal,rsfmri_cor_network.gordon_cingulooperc_network.gordon_default,rsfmri_cor_network.gordon_cingulooperc_network.gordon_dorsalattn,rsfmri_cor_network.gordon_cingulooperc_network.gordon_frontoparietal,rsfmri_cor_network.gordon_cingulooperc_network.gordon_none,rsfmri_cor_network.gordon_cingulooperc_network.gordon_retrosplenialtemporal,rsfmri_cor_network.gordon_cingulooperc_network.gordon_smhand,rsfmri_cor_network.gordon_cingulooperc_network.gordon_smmouth,rsfmri_cor_network.gordon_cingulooperc_network.gordon_salience,rsfmri_cor_network.gordon_cingulooperc_network.gordon_ventralattn,rsfmri_cor_network.gordon_cingulooperc_network.gordon_visual,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_auditory,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_cingulooperc,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_cinguloparietal,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_default,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_dorsalattn,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_frontoparietal,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_none,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_retrosplenialtemporal,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_smhand,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_smmouth,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_salience,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_ventralattn,rsfmri_cor_network.gordon_cinguloparietal_network.gordon_visual,rsfmri_cor_network.gordon_default_network.gordon_auditory,rsfmri_cor_network.gordon_default_network.gordon_cingulooperc,rsfmri_cor_network.gordon_default_network.gordon_cinguloparietal,rsfmri_cor_network.gordon_default_network.gordon_default,rsfmri_cor_network.gordon_default_network.gordon_dorsalattn,rsfmri_cor_network.gordon_default_network.gordon_frontoparietal,rsfmri_cor_network.gordon_default_network.gordon_none,rsfmri_cor_network.gordon_default_network.gordon_retrosplenialtemporal,rsfmri_cor_network.gordon_default_network.gordon_smhand,rsfmri_cor_network.gordon_default_network.gordon_smmouth,rsfmri_cor_network.gordon_default_network.gordon_salience,rsfmri_cor_network.gordon_default_network.gordon_ventralattn,rsfmri_cor_network.gordon_default_network.gordon_visual,rsfmri_cor_network.gordon_dorsalattn_network.gordon_auditory,rsfmri_cor_network.gordon_dorsalattn_network.gordon_cin

## Assemble imaging tables

In [394]:
image_tbls = ["abcd_smrip101", "abcd_smrip201", "abcd_dti_p101", "abcd_dti_p201",
 "mri_rsi_p102", "mri_rsi_p202", "abcd_midr1bwp101", "abcd_midr1bwp201",
 "midr2bwp101", "midr2bwp201", "mrisstr1bw01", "mrisstr2bw01", "nbackr101",
 "nbackr201", "abcd_mrirstv02", "midaparc02", "midaparcp202", 
 "abcd_midasemp101", "abcd_midasemp201", "abcd_midsemp101",
 "abcd_midsemp201", "abcd_midr2semp101", "abcd_midr2semp201",
 "mrisst02", "mrisstsem01", "mrisstr1sem01", "mrisstr2bwsem01",
 "nback_bwroi02", "nbackallsem01", "nbackr1sem01", "nbackr2sem01",
 "dmriqc01", "abcd_mid02", "abcd_sst02", "abcd_mrinback02", "mribrec02",
 "abcd_mrfindings01", "mriqc02", "mriqcp202", "freesqc01"]

In [395]:
non_deskian_tables = ["abcd_mrirstv02", "dmriqc01", "abcd_mid02", "abcd_sst02", "abcd_mrinback02", "mribrec02",
 "abcd_mrfindings01", "mriqc02", "mriqcp202", "freesqc01"]
deskiab_tbls = [it for it in image_tbls if it not in non_deskian_tables]

In [396]:
imgtbls = {}
imglbls = []
for itn in deskiab_tbls:
    imgtbl, imglbl = load_abcd_table(abcd_table_path +itn + '.txt')
    imglbl['source_file'] = itn
    imgtbls[itn] = imgtbl
    imglbls.append(imglbl)

In [397]:
imglbls = pd.concat(imglbls, ignore_index=True)

In [398]:
pd.unique(imgtbl.columns[imgtbl.columns.str.contains('\.')].str.split('_').str[0])

array(['tfmri'], dtype=object)

In [399]:
# get the modality
modalities = pd.unique(imgtbl.columns[imgtbl.columns.str.contains('\.')].str.split('_').str[0])
assert len(modalities) == 1
modality = modalities[0]

In [400]:
imglbls['dat'] = imglbls.name.apply(lambda x: len(x.split('.'))>2)

In [401]:
atlas_guess = imglbls.name[imglbls.name.str.contains('\.') & imglbls.dat].str.split('_').str[-2].unique()
atlas_guess


array(['cort.desikan', 'cort.fuzzy2', 'cort.fuzzy4', 'cort.fuzzy12',
       'subcort.aseg', 'fiber.at'], dtype=object)

In [402]:
# those look like reasonable potential atlasses
# we'll assert that list to make sure we catch any future changes

assert (atlas_guess == np.array(['cort.desikan', 'cort.fuzzy2', 'cort.fuzzy4', 'cort.fuzzy12', 'subcort.aseg', 'fiber.at'])).all()
for atlas in atlas_guess:
    imglbls.loc[imglbls.name.str.contains(atlas), 'atlas'] = atlas

In [403]:
imglbls['roi'] = imglbls.loc[pd.notnull(imglbls.atlas), :].apply(lambda x: x['name'].split(x.atlas + '_')[-1], axis=1)

In [404]:
common_search = (imglbls.groupby('name')[['source_file']]
                  .nunique()
                  .sort_values('source_file', ascending=False)
                  .reset_index()
             )
common_search

,name,source_file
0,src_subject_id,30
1,interview_date,30
2,eventname,30
3,dataset_id,30
4,collection_title,30
5,subjectkey,30
6,gender,30
7,interview_age,30
8,collection_id,30
9,study_cohort_name,30


In [405]:
common_cols = common_search.query('source_file >= 23').name.values

In [406]:
imglbls['common']= imglbls.name.isin(common_cols)

In [407]:
imglbls['dat'] = pd.notnull(imglbls.roi)

In [408]:
imglbls['other'] = imglbls.query('~dat & ~common').name.str.split('_').str[-1]

In [409]:
imglbls.other.unique()

array([nan, 'id', 'visitid', 'rrmdfrlh', 'mean.motion', 'mean.trans',
       'mean.rot', 'rncntelh', 'tr', 'numtrs', 'dof', 'nvols',
       'subthresh.nvols', 'max.motion', 'max.trans', 'max.rot',
       'cdmdftrh', 'clmdfrh', '465', '473', 'tfmrinbr2pscdkracgerh'],
      dtype=object)

In [410]:
# Some of those look ok, others, not so much
tbl_meta = ['id', 'visitid', 'mean.motion', 'mean.trans',
            'mean.rot', 'tr', 'numtrs', 'dof', 'nvols',
            'subthresh.nvols', 'max.motion', 'max.trans', 'max.rot']
#tbl_meta = ['_' + tm for tm in tbl_meta]

### Figure out names to clean up

In [411]:
name_reform = {
    'tfmri_nback_all_465': 'tfmri_nback_all_place_beta_cort.desikan_rostralanteriorcingulate.rh',
    'tfmri_nback_r1sem_465': 'tfmri_nback_run1_place_sem_cort.desikan_rostralanteriorcingulate.rh',
    'tfmrinbackallsem_473': 'tfmri_nback_all_place_sem_cort.desikan_transversetemporal.rh',
    'tfmri_mr1_asrvn_bcdk_cdmdftrh': 'tfmri_mid_run1_antic.small.reward.vs.neutral_beta_cort.desikan_caudalmiddlefrontal.rh',
    'tfmri_mr2_asrvn_bcdk_clmdfrh': 'tfmri_mid_run2_antic.small.reward.vs.neutral_beta_cort.desikan_caudalmiddlefrontal.rh',
    'dmri_dtimdgm_cdsn_rncntelh': 'dmri_dti.md.gm_cort.desikan_rostralanteriorcingulate.lh',
    'smri_t1wcnt_cdk_rrmdfrlh': 'smri_t1w.contrast_cort.desikan_rostralmiddlefrontal.lh',
    'tfmrinbr2pscdkracgerh': 'tfmri_nback_run2_place_sem_cort.desikan_rostralanteriorcingulate.rh'
}
# We'll need to pull surrounding instances to see what the names should be

In [412]:
foo = imglbls.reset_index().groupby('other')

In [413]:
n = 19575
imglbls.loc[n-2:n+2, :]

,name,doc,source_file,dat,atlas,roi,common,other
19573,tfmri_nback_run2_emotion_sem_cort.desikan_lateralorbitofrontal.lh,Standard error of the mean for nBack run 2 emotion condition in APARC ROI lh-lateralorbitofrontal,nbackr2sem01,True,cort.desikan,lateralorbitofrontal.lh,False,NaN
19574,tfmri_nback_run2_emotion_sem_cort.desikan_lingual.lh,Standard error of the mean for nBack run 2 emotion condition in APARC ROI lh-lingual,nbackr2sem01,True,cort.desikan,lingual.lh,False,NaN
19575,tfmri_nback_run2_emotion_sem_cort.desikan_medialorbitofrontal.lh,Standard error of the mean for nBack run 2 emotion condition in APARC ROI lh-medialorbitofrontal,nbackr2sem01,True,cort.desikan,medialorbitofrontal.lh,False,NaN
19576,tfmri_nback_run2_emotion_sem_cort.desikan_middletemporal.lh,Standard error of the mean for nBack run 2 emotion condition in APARC ROI lh-middletemporal,nbackr2sem01,True,cort.desikan,middletemporal.lh,False,NaN
19577,tfmri_nback_run2_emotion_sem_cort.desikan_parahippocampal.lh,Standard error of the mean for nBack run 2 emotion condition in APARC ROI lh-parahippocampal,nbackr2sem01,True,cort.desikan,parahippocampal.lh,False,NaN


In [414]:
foo.groups

{'465': Int64Index([16838, 18648], dtype='int64'),
 '473': Int64Index([17751], dtype='int64'),
 'cdmdftrh': Int64Index([4912], dtype='int64'),
 'clmdfrh': Int64Index([5926], dtype='int64'),
 'dof': Int64Index([ 4217,  5230,  6245,  6954,  7663,  8568,  9473, 10487, 11501,
             12515, 13529, 14238, 14947, 15656, 16365, 17270, 18175, 19080],
            dtype='int64'),
 'id': Int64Index([    1,   762,  1220,  1991,  2428,  3343,  4206,  4528,  5220,
              5542,  6234,  6943,  7652,  8557,  9462,  9784, 10476, 10798,
             11490, 11812, 12504, 12826, 13518, 14227, 14936, 15645, 16354,
             17259, 18164, 19069],
            dtype='int64'),
 'max.motion': Int64Index([ 4221,  5234,  6249,  6958,  7667,  8572,  9477, 10491, 11505,
             12519, 13533, 14242, 14951, 15660, 16369, 17274, 18179, 19084],
            dtype='int64'),
 'max.rot': Int64Index([ 4225,  5238,  6253,  6962,  7671,  8576,  9481, 10495, 11509,
             12523, 13537, 14246, 14955, 15

In [415]:
imglbls[imglbls.doc.str.contains('rostral anterior cingulate')]

,name,doc,source_file,dat,atlas,roi,common,other
1586,dmri_dti.fa.wm_cort.desikan_rostralanteriorcingulate.lh,"Average fractional anisotropy, parcellation of sub-adjacent white matter associated with cortical ROI left-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.lh,False,NaN
1620,dmri_dti.fa.wm_cort.desikan_rostralanteriorcingulate.rh,"Average fractional anisotropy, parcellation of sub-adjacent white matter associated with cortical ROI right-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.rh,False,NaN
1657,dmri_dti.md.wm_cort.desikan_rostralanteriorcingulate.lh,"Mean diffusivity, parcellation of sub-adjacent white matter associated with cortical ROI left-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.lh,False,NaN
1691,dmri_dti.md.wm_cort.desikan_rostralanteriorcingulate.rh,"Mean diffusivity, parcellation of sub-adjacent white matter associated with cortical ROI right-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.rh,False,NaN
1728,dmri_dti.ld.wm_cort.desikan_rostralanteriorcingulate.lh,"Average longitudinal diffusivity, parcellation of sub-adjacent white matter associated with cortical ROI left-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.lh,False,NaN
1762,dmri_dti.ld.wm_cort.desikan_rostralanteriorcingulate.rh,"Average longitudinal diffusivity, parcellation of sub-adjacent white matter associated with cortical ROI right-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.rh,False,NaN
1799,dmri_dti.td.wm_cort.desikan_rostralanteriorcingulate.lh,"Average transverse diffusivity, parcellation of sub-adjacent white matter associated with cortical ROI left-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.lh,False,NaN
1833,dmri_dti.td.wm_cort.desikan_rostralanteriorcingulate.rh,"Average transverse diffusivity, parcellation of sub-adjacent white matter associated with cortical ROI right-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.rh,False,NaN
1870,dmri_dti.fa.gm_cort.desikan_rostralanteriorcingulate.lh,"Average fractional anisotropy, parcellation of cortical gray matter associated with cortical ROI left-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.lh,False,NaN
1904,dmri_dti.fa.gm_cort.desikan_rostralanteriorcingulate.rh,"Average fractional anisotropy, parcellation of cortical gray matter associated with cortical ROI right-rostral anterior cingulate",abcd_dti_p101,True,cort.desikan,rostralanteriorcingulate.rh,False,NaN


### apply name cleaning learned above

In [416]:
name_report = imglbls.loc[imglbls.name.isin(name_reform.keys()), ['name', 'source_file']].copy()

In [417]:
name_report['fixed_name'] = name_report.name.apply(lambda x: name_reform[x])

In [418]:
name_report.loc[:, ['source_file', 'name', 'fixed_name']]

,source_file,name,fixed_name
677,abcd_smrip101,smri_t1wcnt_cdk_rrmdfrlh,smri_t1w.contrast_cort.desikan_rostralmiddlefrontal.lh
1941,abcd_dti_p101,dmri_dtimdgm_cdsn_rncntelh,dmri_dti.md.gm_cort.desikan_rostralanteriorcingulate.lh
4912,abcd_midr1bwp201,tfmri_mr1_asrvn_bcdk_cdmdftrh,tfmri_mid_run1_antic.small.reward.vs.neutral_beta_cort.desikan_caudalmiddlefrontal.rh
5926,midr2bwp201,tfmri_mr2_asrvn_bcdk_clmdfrh,tfmri_mid_run2_antic.small.reward.vs.neutral_beta_cort.desikan_caudalmiddlefrontal.rh
16838,nback_bwroi02,tfmri_nback_all_465,tfmri_nback_all_place_beta_cort.desikan_rostralanteriorcingulate.rh
17751,nbackallsem01,tfmrinbackallsem_473,tfmri_nback_all_place_sem_cort.desikan_transversetemporal.rh
18648,nbackr1sem01,tfmri_nback_r1sem_465,tfmri_nback_run1_place_sem_cort.desikan_rostralanteriorcingulate.rh
19553,nbackr2sem01,tfmrinbr2pscdkracgerh,tfmri_nback_run2_place_sem_cort.desikan_rostralanteriorcingulate.rh


In [419]:
# Just found the mapping file that will have the real fixed names as opposed to my guesses, lets use that instead
abcd_mapping = pd.read_csv('/data/MBDU/nielsond/abcd_combat/nielson_abcd_2018/ABCDstudyDEAP2.0/results/ABCD_Release_Notes_Data_Release_2.0/21. ABCD_Release_1.1_mapping_file.csv')

In [420]:
# turns out I got all the names right
name_report.name.apply(lambda x:abcd_mapping.loc[abcd_mapping['ABCD Release 1.1 name'] == x, 'DEAP name'].values[0])==name_report.fixed_name

677      True
1941     True
4912     True
5926     True
16838    True
17751    True
18648    True
19553    True
dtype: bool

In [421]:
for i,row in name_report.iterrows():
    imgtbls[row.source_file] = imgtbls[row.source_file].rename(columns={row['name']:row.fixed_name}).copy()

In [422]:
imglbls.name = imglbls.name.replace(name_reform)

### now that names are clean, get metadata again

In [423]:
for atlas in atlas_guess:
    imglbls.loc[imglbls.name.str.contains(atlas), 'atlas'] = atlas
imglbls['roi'] = imglbls.loc[pd.notnull(imglbls.atlas), :].apply(lambda x: x['name'].split(x.atlas + '_')[-1], axis=1)
imglbls['common']= imglbls.name.isin(common_cols)
imglbls['dat'] = pd.notnull(imglbls.roi)


In [424]:
imglbls['long_form_name'] = imglbls.loc[imglbls.common, 'name']

In [425]:
imglbls.head()

,name,doc,source_file,dat,atlas,roi,common,other,long_form_name
0,collection_id,collection_id,abcd_smrip101,False,NaN,NaN,True,NaN,collection_id
1,abcd_smrip101_id,abcd_smrip101_id,abcd_smrip101,False,NaN,NaN,False,id,NaN
2,dataset_id,dataset_id,abcd_smrip101,False,NaN,NaN,True,NaN,dataset_id
3,subjectkey,The NDAR Global Unique Identifier (GUID) for research subject,abcd_smrip101,False,NaN,NaN,True,NaN,subjectkey
4,src_subject_id,Subject ID how it's defined in lab/project,abcd_smrip101,False,NaN,NaN,True,NaN,src_subject_id


In [426]:
imglbls['meta'] = False
for tm in tbl_meta:
    tm_ind = imglbls.name.str.contains('_' + tm)
    imglbls.loc[tm_ind & ~imglbls.common, 'meta'] = True
    imglbls.loc[tm_ind & ~imglbls.common, 'long_form_name' ] = 'tbl_' + tm

In [427]:
imglbls.head(100)

,name,doc,source_file,dat,atlas,roi,common,other,long_form_name,meta
0,collection_id,collection_id,abcd_smrip101,False,NaN,NaN,True,NaN,collection_id,False
1,abcd_smrip101_id,abcd_smrip101_id,abcd_smrip101,False,NaN,NaN,False,id,tbl_id,True
2,dataset_id,dataset_id,abcd_smrip101,False,NaN,NaN,True,NaN,dataset_id,False
3,subjectkey,The NDAR Global Unique Identifier (GUID) for research subject,abcd_smrip101,False,NaN,NaN,True,NaN,subjectkey,False
4,src_subject_id,Subject ID how it's defined in lab/project,abcd_smrip101,False,NaN,NaN,True,NaN,src_subject_id,False
5,interview_date,Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY,abcd_smrip101,False,NaN,NaN,True,NaN,interview_date,False
6,interview_age,Age in months at the time of the interview/test/sampling/imaging.,abcd_smrip101,False,NaN,NaN,True,NaN,interview_age,False
7,gender,Sex of the subject,abcd_smrip101,False,NaN,NaN,True,NaN,gender,False
8,eventname,The event name for which the data was collected,abcd_smrip101,False,NaN,NaN,True,NaN,eventname,False
9,smri_visitid,Visit name,abcd_smrip101,False,NaN,NaN,False,visitid,tbl_visitid,True


### Now that we've dealt with the other columns, lets see what we can parse out of the stuff preceeding the atlas

In [428]:
# Get the modality
# Does the first tag of the dat column only give us the expected modalities
modalities = pd.unique(imglbls.loc[imglbls.dat, :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[0], axis=1))
assert (modalities == np.array(['smri', 'dmri', 'tfmri'])).all()

In [429]:
# looks good, we'll assign that and move on
imglbls['modality'] = imglbls.loc[imglbls.dat, :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[0], axis=1)


In [430]:
# Make sure that the smri and dmri each have three fields and that the tfmri has 6
assert pd.unique(imglbls.loc[imglbls.modality == 'smri', :].apply(lambda x: len(x['name'].split(x.atlas + '_')[0].split('_')), axis=1))[0] == 3
assert pd.unique(imglbls.loc[imglbls.modality == 'dmri', :].apply(lambda x: len(x['name'].split(x.atlas + '_')[0].split('_')), axis=1))[0] == 3
assert pd.unique(imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: len(x['name'].split(x.atlas + '_')[0].split('_')), axis=1))[0] == 6
# make sure the last field is blank, corresponding to the trailing underscore
assert pd.unique(imglbls.loc[imglbls.dat, :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[-1], axis=1))[0] == ''

In [431]:
# smri labels
smri_labels = pd.unique(imglbls.loc[imglbls.modality == 'smri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[1], axis=1))
assert (smri_labels == np.array(['thick', 'sulc', 'area', 'vol', 't1w.white02', 't1w.gray02',
                                't1w.contrast', 't2w.white02', 't2w.gray02', 't2w.contrast', 't1w',
                                't2w'])).all()
imglbls['metric'] = imglbls.loc[imglbls.modality == 'smri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[1], axis=1)

In [432]:
# dti labels
dti_labels = pd.unique(imglbls.loc[imglbls.modality == 'dmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[1], axis=1))
assert (dti_labels == np.array(['dti.fa', 'dti.md', 'dti.ld', 'dti.td', 'dti.vol',
       'dti.fa.wm', 'dti.md.wm', 'dti.ld.wm', 'dti.td.wm', 'dti.fa.gm',
       'dti.md.gm', 'dti.ld.gm', 'dti.td.gm', 'dti.fa.gwc', 'dti.md.gwc',
       'dti.ld.gwc', 'dti.td.gwc', 'rsi.n0', 'rsi.n0s2', 'rsi.nd',
       'rsi.nds2', 'rsi.nt', 'rsi.nts2', 'rsi.vol', 'rsi.n0.wm',
       'rsi.n0s2.wm', 'rsi.nd.wm', 'rsi.nds2.wm', 'rsi.nt.wm',
       'rsi.nts2.wm', 'rsi.n0.gm', 'rsi.n0s2.gm', 'rsi.nd.gm',
       'rsi.nds2.gm', 'rsi.nt.gm', 'rsi.nts2.gm', 'rsi.n0.gwc',
       'rsi.n0s2.gwc', 'rsi.nd.gwc', 'rsi.nds2.gwc', 'rsi.nt.gwc',
       'rsi.nts2.gwc'])).all()
imglbls.loc[imglbls.modality == 'dmri', 'metric'] = imglbls.loc[imglbls.modality == 'dmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[1], axis=1)


In [433]:
# fmri data has more levels of organization
for ii in range(6):
    print((ii, pd.unique(imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[ii], axis=1))))

(0, array(['tfmri'], dtype=object))
(1, array(['mid', 'sst', 'nback'], dtype=object))
(2, array(['run1', 'run2', 'all'], dtype=object))
(3, array(['antic.reward.vs.neutral', 'antic.loss.vs.neutral',
       'reward.pos.vs.neg.feedback', 'loss.pos.vs.neg.feedback',
       'antic.large.reward.vs.neutral', 'antic.small.reward.vs.neutral',
       'antic.large.vs.small.reward', 'antic.large.loss.vs.neutral',
       'antic.small.loss.vs.neutral', 'antic.large.vs.small.loss',
       'correct.go.vs.fixation', 'correct.stop.vs.correct.go',
       'incorrect.stop.vs.correct.go', 'any.stop.vs.correct.go',
       'correct.stop.vs.incorrect.stop', 'incorrect.go.vs.correct.go',
       'incorrect.go.vs.incorrect.stop', '0.back', '2.back', 'place',
       'emotion', '2.back.vs.0.back', 'face.vs.place',
       'emotion.vs.neutface', 'negface.vs.neutface',
       'posface.vs.neutface'], dtype=object))
(4, array(['beta', 'sem'], dtype=object))
(5, array([''], dtype=object))


In [434]:
# for fmri the levels of organization appear to be modality, task, run, contrast, metric
tasks = pd.unique(imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[1], axis=1))
assert (tasks == np.array(['mid', 'sst', 'nback'])).all()
imglbls['task'] = imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[1], axis=1)

runs = pd.unique(imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[2], axis=1))
assert (runs == np.array(['run1', 'run2', 'all'])).all()
imglbls['run'] = imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[2], axis=1)

contrasts = pd.unique(imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[3], axis=1))
assert (contrasts == np.array(['antic.reward.vs.neutral', 'antic.loss.vs.neutral',
       'reward.pos.vs.neg.feedback', 'loss.pos.vs.neg.feedback',
       'antic.large.reward.vs.neutral', 'antic.small.reward.vs.neutral',
       'antic.large.vs.small.reward', 'antic.large.loss.vs.neutral',
       'antic.small.loss.vs.neutral', 'antic.large.vs.small.loss',
       'correct.go.vs.fixation', 'correct.stop.vs.correct.go',
       'incorrect.stop.vs.correct.go', 'any.stop.vs.correct.go',
       'correct.stop.vs.incorrect.stop', 'incorrect.go.vs.correct.go',
       'incorrect.go.vs.incorrect.stop', '0.back', '2.back', 'place',
       'emotion', '2.back.vs.0.back', 'face.vs.place',
       'emotion.vs.neutface', 'negface.vs.neutface',
       'posface.vs.neutface'])).all()
imglbls['contrast'] = imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[3], axis=1)

metrics = pd.unique(imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[4], axis=1))
assert (metrics == np.array(['beta', 'sem'])).all()
imglbls.loc[imglbls.modality == 'tfmri', 'metric'] = imglbls.loc[imglbls.modality == 'tfmri', :].apply(lambda x: x['name'].split(x.atlas + '_')[0].split('_')[4], axis=1)



In [435]:
# make sure there aren't any nans on the modality, task, run, contrast, or metric columns for the corresponding tasks
assert imglbls.loc[imglbls.dat, 'modality'].isnull().sum() == 0
assert imglbls.loc[imglbls.dat, 'metric'].isnull().sum() == 0
assert imglbls.loc[imglbls.dat, 'atlas'].isnull().sum() == 0
assert imglbls.loc[imglbls.dat, 'roi'].isnull().sum() == 0
assert imglbls.loc[imglbls.modality == "fmri", 'task'].isnull().sum() == 0
assert imglbls.loc[imglbls.modality == "fmri", 'run'].isnull().sum() == 0
assert imglbls.loc[imglbls.modality == "fmri", 'contrast'].isnull().sum() == 0


In [436]:
imglbls.groupby('atlas').roi.nunique()

atlas
cort.desikan    74
cort.fuzzy12    24
cort.fuzzy2      4
cort.fuzzy4      8
fiber.at        42
subcort.aseg    46
Name: roi, dtype: int64

In [437]:
imglbls.loc[imglbls.dat, 'long_form_name'] = imglbls.loc[imglbls.dat, 'atlas'] + '__' + imglbls.loc[imglbls.dat, 'roi']

In [438]:
assert imglbls.long_form_name.isnull().sum() == 0

## Now that we've got the labels figured out, lets build the long table

In [439]:
gblist =['modality', 'metric', 'task', 'run', 'contrast']
gb = imglbls.query('dat').groupby(gblist)
for x,df in gb:
    break

In [440]:
def make_long(modality, labels, tables, new_id_vars=['modality', 'metric']):
    '''
    Parameters
    ----------
    modality: string
        Name of the modality to extract and make long
    labels: DataFrame
        Information about the column names in the existing tables
        and the appropriate names for those columns in the long table
    tables: Dictonary of DataFrames with source file as key
        The tables from which information will be extracted
    new_id_vars: list of strings, default ['modality', 'metric']
        Names of new id variables described in labels that will
        be added to rows of long table
    
    Returns
    -------
    longtbl: DataFrame
        Partially melted dataframe that now has new_id_vars added
    
        '''
    longtbl = []
    gb = labels.query('dat & modality == @modality').groupby(new_id_vars)
    for x, df in gb:
        # For each set of unique new_id_vars, extract the appropriate
        # information from each source table and merge them
        for sfi, sf in enumerate(np.unique(df.source_file.values)):
            # Combine the id columns present for this source file
            # with the value columns present in this group of new_id_vars
            new_cols = pd.concat([labels.query('source_file == @sf & ~dat'),
                                     df.query('source_file == @sf')])
            
            # Extract the mapping from orig column names to long names
            col_map = {nn:lfn for nn,lfn 
                       in new_cols.loc[:, ['name', 'long_form_name']].values}
            
            # Create map between new_id_var names and values
            id_var_map = {gbi:xi for gbi,xi in zip(new_id_vars,x)}
            
            # Pull the columns, rename them, add new_id_vars
            inner_tmp = (tables[sf].loc[:, new_cols.name]
                                   .rename(columns=col_map)
                                   .assign(**id_var_map)
                                   .assign(source_file=sf))
            
            if sfi ==  0:
                tmp_df = inner_tmp
            else:
                # Figure out which columns are present to merge on
                merge_on = set(tmp_df.columns).intersection(inner_tmp.columns)
                # tbl_id and source_file depend on source_file
                # so don't merge on them
                merge_on.discard('tbl_id')
                merge_on.discard('source_file')
                # merge the tables
                tmp_df = tmp_df.merge(inner_tmp,
                                      on=list(merge_on),
                                      how='outer',
                                      suffixes=('', '_y'))
        longtbl.append(tmp_df)
    
    longtbl = pd.concat(longtbl, sort=False, ignore_index=True)
    return longtbl

In [441]:
longtbls = []
for modality in modalities:
    # Have to separate by modality because pandas
    # drops rows that are nan in one of the group by columns
    if modality == 'tfmri':
        new_id_vars = ['modality', 'metric', 'task', 'run', 'contrast']
    else:
        new_id_vars = ['modality', 'metric']
    
    longtbls.append(make_long(modality, imglbls, imgtbls, new_id_vars))
#longtbl = pd.concat(longtbls, sort=False, ignore_index=True)

In [442]:
longtbl = pd.concat(longtbls[:2], sort=False, ignore_index=True)

In [443]:
longtbl = pd.concat([longtbl, longtbls[2]], sort=False, ignore_index=True)

# Now we'll merge in balancing

In [444]:
# For right now at least, it appears that interview date has been set to a single one per subject
# That can act as the visit id
assert longtbl[longtbl.tbl_visitid.notnull()].groupby(['subjectkey'])['interview_date'].nunique().max() == 1

In [445]:
longtbl = longtbl.merge(for_balancing.loc[:,['subjectkey', 'interview_date', 'site_id_l',
       'mri_info_visitid', 'mri_info_manufacturer', 'mri_info_manufacturersmn',
       'mri_info_deviceserialnumber', 'mri_info_magneticfieldstrength',
       'mri_info_softwareversion']], how='left',
          on=['subjectkey','interview_date'], indicator=True)

In [446]:
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

# Fix event name

In [447]:
longtbl = longtbl.merge(for_balancing.loc[:,['subjectkey', 'interview_date', 'eventname']], 
                        how='left',
                        on=['subjectkey','interview_date'],
                        indicator=True,
                        suffixes=('', '_mergetmp'))
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl.eventname.fillna(longtbl.eventname_mergetmp, inplace=True)
longtbl.drop('eventname_mergetmp', axis=1, inplace=True)

In [448]:
longtbl.columns[longtbl.isnull().sum() == longtbl.shape[0]]

Index(['subcort.aseg__lesion.lh', 'subcort.aseg__lesion.rh'], dtype='object')

# Now merge in qc tables

## connectivity

In [449]:
fsqc, fsqclbl = load_abcd_table(abcd_table_path + 'freesqc01.txt')

In [450]:
con = con.merge(fsqc.loc[:, ['subjectkey', 'interview_date', 'eventname', 'fsqc_qc']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert con.groupby('_merge')['collection_id'].count()['both'] == len(con)
con.drop('_merge', axis=1, inplace=True)

In [451]:
con['qc_ok'] = (con['rsfmri_cor_network.gordon_ntpoints'] > 375) & (con.fsqc_qc == 1)

## Everything else

In [452]:
dmriqc, dmriqclbl = load_abcd_table(abcd_table_path + 'dmriqc01.txt')
mriqc, mriqclbl = load_abcd_table(abcd_table_path + 'mriqcrp102.txt') # 
mriqc2, mriqclbl = load_abcd_table(abcd_table_path + 'mriqcrp202.txt') # 
mriqc3, mriqclbl = load_abcd_table(abcd_table_path + 'mriqcrp302.txt') # 
midbeh, _ = load_abcd_table(abcd_table_path + 'abcd_mid02.txt')
nbbeh, _ = load_abcd_table(abcd_table_path + 'abcd_mrinback02.txt')
sstbeh, _ = load_abcd_table(abcd_table_path + 'abcd_sst02.txt')
mrfind, _ = load_abcd_table(abcd_table_path + 'abcd_mrfindings01.txt')

In [453]:
longtbl = longtbl.merge(fsqc.loc[:, ['subjectkey', 'interview_date', 'eventname', 'fsqc_qc']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
noimaging = longtbl.groupby(['_merge'])[['subjectkey']].count().loc['both', 'subjectkey']
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [454]:
longtbl = longtbl.merge(dmriqc.loc[:, ['subjectkey', 'interview_date', 'eventname', 'dmri_dti_postqc_qc']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
print("long stuff = %d" % longtbl.groupby('_merge')['collection_id'].count()['both'])
print("longtbl = %d" % len(longtbl))
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

long stuff = 2553815
longtbl = 2553911


In [455]:
longtbl = longtbl.merge(mriqc.loc[:, ['subjectkey', 'interview_date', 'eventname', 'iqc_t2_ok_ser']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [456]:
longtbl = longtbl.merge(mriqc2.loc[:, ['subjectkey', 'interview_date', 'iqc_sst_ok_ser', 'iqc_nback_ok_ser', 'iqc_mid_ok_ser']],
                how='left',
                on=['subjectkey','interview_date'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [457]:
longtbl = longtbl.merge(midbeh.loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_mid_beh_perform.flag']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(nbbeh.loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_nback_beh_perform.flag']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(sstbeh.loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_sst_beh_perform.flag']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [458]:
longtbl = longtbl.merge(imgtbls['midaparc02'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_mid_all_beta_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(imgtbls['abcd_midasemp101'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_mid_all_sem_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)


longtbl = longtbl.merge(imgtbls['mrisst02'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_sst_all_beta_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(imgtbls['mrisstsem01'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_sst_all_sem_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)


longtbl = longtbl.merge(imgtbls['nback_bwroi02'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_nback_all_beta_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(imgtbls['nbackallsem01'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_nback_all_sem_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [459]:
longtbl = longtbl.merge(mrfind.loc[:, ['subjectkey', 'interview_date', 'eventname', 'mrif_score', 'mrif_hydrocephalus', 'mrif_herniation']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
longtbl = longtbl.query("_merge == 'both'")
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [460]:
longtbl['mr_findings_ok'] = (longtbl['mrif_score'] < 3)  & (longtbl['mrif_hydrocephalus'] == 'no') &  (longtbl['mrif_herniation'] == 'no')


In [461]:
# Default everyting to qc_ok as False
longtbl['qc_ok'] = False


In [462]:
# QC qualifications for smri

t1_metrics = ['area', 'sulc', 't1w', 't1w.contrast', 't1w.gray02', 't1w.white02', 'thick', 'vol']
t2_metrics = ['t2w', 't2w.contrast', 't2w.gray02', 't2w.white02']


longtbl.loc[((longtbl.modality == 'smri') 
             & (longtbl.metric.isin(t1_metrics))
             & (longtbl.fsqc_qc == 1)), 'qc_ok'] = True


longtbl.loc[((longtbl.modality == 'smri') 
             & (longtbl.metric.isin(t2_metrics))
             & (longtbl.fsqc_qc == 1)
             & (longtbl.iqc_t2_ok_ser > 0)), 'qc_ok'] = True

In [463]:
# QC qualifications for dmri
# Only data in fsrois depends on a good fs segmentation, but since I aggregated metrics the way I did, this ends up being all metrics
fs_roi_rows = longtbl.loc[longtbl.modality == 'dmri', imglbls.loc[imglbls.atlas != 'fiber.at', 'long_form_name'].unique()].notnull().sum(1) > 0

longtbl.loc[((longtbl.modality == 'dmri') 
             & fs_roi_rows
             & (longtbl.fsqc_qc == 1)
             & (longtbl.dmri_dti_postqc_qc > 0)), 'qc_ok'] = True

In [464]:
# QC for MID task
longtbl.loc[((longtbl.modality == 'tfmri') 
             & (longtbl.task == 'mid')
             & (longtbl.fsqc_qc == 1)
             & (longtbl.iqc_mid_ok_ser > 0)
             & (longtbl['tfmri_mid_beh_perform.flag'] == 1)
             & (longtbl.tfmri_mid_all_beta_dof > 200)
             & (longtbl.tfmri_mid_all_sem_dof > 200)), 'qc_ok'] = True

# QC for SST
longtbl.loc[((longtbl.modality == 'tfmri') 
             & (longtbl.task == 'sst')
             & (longtbl.fsqc_qc == 1)
             & (longtbl.iqc_sst_ok_ser > 0)
             & (longtbl['tfmri_sst_beh_perform.flag'] == 1)
             & (longtbl.tfmri_sst_all_beta_dof > 200)
             & (longtbl.tfmri_sst_all_sem_dof > 200)), 'qc_ok'] = True

# QC for nBack
longtbl.loc[((longtbl.modality == 'tfmri') 
             & (longtbl.task == 'nback')
             & (longtbl.fsqc_qc == 1)
             & (longtbl.iqc_nback_ok_ser > 0)
             & (longtbl['tfmri_nback_beh_perform.flag'] == 1)
             & (longtbl.tfmri_nback_all_beta_dof > 200)
             & (longtbl.tfmri_nback_all_sem_dof > 200)), 'qc_ok'] = True

In [ ]:
con.to_pickle(data_dir/'con_2.0.pkz')
longtbl.to_pickle(data_dir/'long_table_2.0.pkz')